In [3]:
import numpy as np
import pandas as pd
import warnings
import cvxopt
import copy
import scipy
from scipy.optimize import minimize
from scipy.optimize import basinhopping
from cvxopt import matrix
warnings.filterwarnings('ignore')

cvxopt.solvers.qp(
    P,
    q,
    G=None,
    h=None,
    A=None,
    b=None,
    solver=None,
    kktsolver=None,
    initvals=None,
   **kwargs)

Docstring:
Solves a quadratic program

    minimize    (1/2)*x'*P*x + q'*x
    subject to  G*x <= h
                A*x = b.

由二次规划的一般形式可知，需要将我们的损失函数转化成这样的一般形式才能代入cvxopt.solvers.qp函数内进行求解，所以

第一步：需要找出P矩阵和q矩阵分别是什么(@代表矩阵相乘, .T代表矩阵的转置)

(1) MPC的损失函数为：J(Uk) = (Xk - Rk).T @ Q_bar @ (Xk - Rk) + Uk.T @ R_bar @ Uk；

(2) 损失函数中含有Xk和Uk，所以需要推导Xk和Uk之间的关系：Xk = M @ xk + C @ Uk 将其代入损失函数J(Uk)中；

(3) J(Uk) = (M @ xk + C @ Uk - Rk).T @ Q_bar @ (M @ xk + C @ Uk - Rk) + Uk.T @ R_bar @ Uk, 经过一系列的代数运算可以将

J(Uk)化成二次规划的形式：J(Uk) = Uk.T @ [C.T @ Q_bar @ C + R_bar] @ Uk + 2*(C.T @ Q_bar @ E).T @ Uk + E.T @ Q_bar @ E

其中E.T @ Q_bar @ E为常数项，在优化时不用代入，于是J(Uk)就转化成关于只求解Uk的二次规划形式，其中的很多矩阵需要在程序内部中去定义，属于中间计算要用到的矩阵。

第二步：第一步计算出的Uk的个数会与预测horizon有关，但是只取Uk的第一个元素。

In [4]:
class LinearModelPredControl:
    
    def __init__(self, A, B, Q, F, R, N, input_, output_, target_):
        self.A = A
        self.B = B
        self.Q = Q
        self.F = F
        self.R = R
        self.N = N
        
        self.input_ = input_
        self.output_ = output_
        self.target_ = target_
        
        self.ref = np.mat(self.target_)
        self.output = np.mat(self.output_).T
        
        self.n = self.A.shape[0]
        self.p = self.B.shape[1]
        self.result = pd.DataFrame()
        
    def MPC_cal_Matrices(self, A, B, Q, R, F, N):
        ## 计算M矩阵
        M = np.mat(np.eye(self.n))
        for i in range(self.N):
            A_mat = self.A**(i + 1)
            M = np.vstack([M, A_mat])

        ## 计算C矩阵，先计算C矩阵的最后填满，再往上填充
        C = np.mat(np.zeros([(self.N + 1)*self.n, self.N*self.p]))
        A_B_cals = []
        J, K = (self.A @ self.B).shape
        m = 0
        for j in range(self.N):
            C[-J:, m:(m + K)] = np.hstack([self.A**(self.N - 1 - j) @ self.B])
            m = m + K
            A_B_cals.append(self.A**(self.N - 1 - j) @ self.B)

        ## C矩阵往上填充
        tmp_len = len(A_B_cals)
        for z in range(1, tmp_len): 
            col_num = len(A_B_cals[z:]) * K
            C[-(z + 1)*J:(-z*J), 0:col_num] = np.hstack(A_B_cals[z:])

        Q_bar = np.kron(np.eye(self.N), self.Q)
        Q_bar = scipy.linalg.block_diag(Q_bar, self.F)
        R_bar = np.kron(np.eye(self.N), self.R)
#         print("Q_bar:", Q_bar.shape)
#         print("R_bar:", R_bar.shape)
        H = (C.T @ Q_bar) @ C + R_bar
        return H, M, C, Q_bar
    
    def MPC_solve_uk(self):
        H, M, C, Q_bar = self.MPC_cal_Matrices(A = self.A, B = self.B, Q = self.Q, R = self.R, F = self.F, N = self.N)
        ref = np.mat(np.array([self.ref for _ in range(self.N + 1)]).flatten()).T

        PP = matrix(2*H)

        E = np.dot(M, self.output) - ref
        qq1 = (C.T @ Q_bar) @ E
        qq = matrix(2 * qq1)

        result = cvxopt.solvers.qp(P = PP, q = qq)
        u_pred = list(result['x'])
        pred_first = u_pred[:self.p] 
#         print(u_pred)
#         print(pred_first)
#         print("*"*50)
        
        return pred_first

    @staticmethod
    def predict(x, opt_u):
        opt_u = np.mat(opt_u).T
        x = A @ x + B @ opt_u
        return x

    def run(self):
        reference = np.mat(np.array([self.ref for _ in range(self.N + 1)]).flatten())
        opt_u = self.MPC_solve_uk()
        y_pred = self.predict(self.output, opt_u) 
#         print(opt_u)
        return opt_u, y_pred
    
    def get_final_results(self):
        opt_u, y_pred = self.run()
        
        res = [opt_u[i] + self.input_[i] for i in range(0, len(self.input_))]
        res = np.array(res).reshape(1, self.n) 
        res = np.round(res, 2)
        res = pd.DataFrame(data = res, columns = ["next" + str(i) for i in range(1, self.n + 1)])
        self.result = pd.concat([self.result, res], axis = 0)
        return self.result

In [ ]:
## data = read.csv("../data.csv")

In [49]:
data_input = data.iloc[:, 0:7] 
data_output = data.iloc[:, 7:14]
data_next_input = data.iloc[:, 14:21]
data_target = data.iloc[:, 21:28]

In [59]:
A = np.mat(np.eye(7))
B = np.mat([[-2, -1, -2, 0, 0, 0, 0],
              [-2, -7, -1, -2, 0, 0, 0],
              [-2, 0, -2, -1, 0, 0, 0],
              [-2, 0, 0, -3, -2, 0, 0],
              [-2, 0, 0, 0, -11, -2, 0],
              [-2, 0, 0, 0, -2, -3, -2],
              [-2, 0, 0, 0, 0, -2, -1]])
n = A.shape[0]
p = B.shape[1]

Q = np.mat(np.eye(n))  # n x n
F = np.mat(np.eye(n))  # n x n
R = np.mat(np.eye(p))   # p x p 
N = 5

In [62]:
i = 1
input_ = data_input.iloc[i].values
output_ = data_output.iloc[i].values
target_ = data_target.iloc[i].values

LMPC = LinearModelPredControl(A, B, Q, F, R, N, input_, output_, target_)
opt_u, y_pred = LMPC.run()
res = LMPC.get_final_results()
result = pd.concat([result, res], axis=0)
result = result.reset_index(drop=True)  

In [38]:
result

,next1,next2,next3,next4,next5,next6,next7
0,19.03,146.88,168.81,174.24,182.80,162.40,201.74
1,16.42,157.64,168.60,169.95,182.80,159.45,210.32
2,14.53,158.88,170.13,170.74,180.84,159.61,215.51
3,17.61,157.77,169.07,176.88,186.93,164.27,206.82
4,18.86,148.57,169.21,174.44,180.85,164.75,197.68
5,19.06,156.49,175.19,181.78,186.39,152.99,240.84
